# Only for Window User

Step 1: Install a Linux Distribution in WSL

Step 2: Install Ansible in your Ubuntu WSL Distribution

Type the following command and press Enter:

#Command

Bash

--------------------->sudo apt update

3.After the update completes, install Ansible itself.
Type the following command and press Enter:

    
#Command

Bash

-------------->sudo apt install ansible -y

4.Finally, let's verify that Ansible has been installed correctly.
Type the following command and press Enter:

#Bash Command

---------------->ansible --version

You should see output similar to this (the exact version numbers might differ, which is fine):

ansible [core 2.15.5]
  config file = None
  configured module search path = ['/home/youruser/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /usr/lib/python3/dist-packages/ansible
  ansible collection location = /home/youruser/.ansible/collections:/usr/share/ansible/collections
  executable location = /usr/bin/ansible
  python version = 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0] (/usr)
  jinja version = 3.0.3
  libyaml collection = True

# Prepare a Target Machine (Managed Node)

Fantastic! You now have Ansible successfully installed in your WSL Ubuntu environment. This means your Windows machine is ready to act as an Ansible Control Node.

Step 3: Prepare a Target Machine (Managed Node)
Now that you have Ansible installed, you need a machine for Ansible to manage. This is called a Managed Node or Target Host.

For a basic configuration, you have two primary options:

A separate Linux Virtual Machine (VM): This is the ideal and most common setup for learning and production. It provides a clear separation between your Ansible control node and the machine it's managing.


Your Windows 11 host machine itself: You can actually configure your Windows 11 machine from your WSL Ubuntu instance, but it requires a bit more initial setup on the Windows side.

# Prepare Your AWS EC2 Target Machine

I.Identify the Target EC2 Instance:

Go to your AWS EC2 Dashboard.

Note down the Public IPv4 address of one of your running Linux instances. Let's call this <YOUR_EC2_PUBLIC_IP>.

Note down the private key file (.pem file) associated with that EC2 instance. This is the key pair you downloaded when you launched the instance. You'll need to transfer this to your WSL environment.

II.Transfer your AWS Private Key (.pem file) to WSL:

Locate your .pem file on your Windows machine (e.g., C:\Users\YourUser\Downloads\my-key-pair.pem).

Open your WSL Ubuntu terminal.

Create a secure directory for your SSH keys in WSL:

In [1]:
mkdir -p ~/.ssh
chmod 700 ~/.ssh

SyntaxError: invalid syntax (490141117.py, line 1)

III.Copy the .pem file from Windows to your WSL's ~/.ssh directory:

The easiest way to do this is to navigate to the Windows file path from within WSL. Windows drives are mounted under /mnt/. So, if your key is in C:\Users\YourUser\Downloads\my-key-pair.pem:

Bash Command

--------------------->cp /mnt/c/Users/YourUser/Downloads/my-key-pair.pem ~/.ssh/

# Replace YourUser with your actual Windows username and my-key-pair.pem with your key file's name.

IV.Set the correct permissions for the private key in WSL: SSH requires private keys to have very strict permissions (only readable by the owner).

Bash Command

-------------->chmod 400 ~/.ssh/my-key-pair.pem
Again, replace my-key-pair.pem with your key file's name.

Ensure SSH Connectivity (Test from WSL):

Most AWS Linux AMIs (like Amazon Linux 2, Ubuntu) come with SSH server installed and Python 3 pre-installed.

Test SSH connectivity from your WSL Ubuntu terminal to your EC2 instance:

Bash COMMAND

----------------------->ssh -i ~/.ssh/my-key-pair.pem ec2-user@<YOUR_EC2_PUBLIC_IP>

(If you used an Ubuntu AMI, the default user might be ubuntu instead of ec2-user. If you used a Red Hat/CentOS AMI, it might be ec2-user or centos.)

Replace my-key-pair.pem and <YOUR_EC2_PUBLIC_IP> with your actual values.

If this connects successfully (you'll see a command prompt for your EC2 instance), type exit to return to your WSL terminal. If it fails, double-check your IP, key path, permissions, and security group settings in AWS (ensure port 22 is open to your IP or 0.0.0.0/0 for testing).

------------>When you get the message "permanently added to the list of known hosts," it means:

Your SSH connection was successful. You were able to reach the EC2 instance via SSH from your WSL Ubuntu terminal.

SSH key authentication worked. Your ~/.ssh/my-key-pair.pem file was correctly used to authenticate.

The host key of the EC2 instance was saved. This means your SSH client (on WSL) now trusts that specific EC2 instance, and you won't get a "The authenticity of host 'X.X.X.X' can't be established" warning again for that IP.

You should now be inside your EC2 instance's terminal (your prompt will likely change to something like ec2-user@ip-XXX-XXX-XXX-XXX:~$ or similar).

Please type exit and press Enter to return to your WSL Ubuntu terminal.

# Please type exit and press Enter to return to your WSL Ubuntu terminal.

# Create an Ansible Project Directory and Inventory File


Ansible needs to know which machines it's supposed to manage. This information is stored in an inventory file. It's also good practice to keep your Ansible files organized in a "project" directory.

In your WSL Ubuntu terminal, make sure you are in your home directory (~). You can type cd ~ and press Enter to ensure this.

1.Create a new directory for your Ansible project:

#Bash Command

-------------->mkdir my_aws_ansible_project

2.Navigate into your new project directory:

#Bash Command

------------->cd my_aws_ansible_project

3.Create your inventory file. We'll name it inventory.ini (the .ini extension is a common format for simple inventories):

Bash Command

-------------------->nano inventory.ini

4.Add the following lines to inventory.ini:

Ini, TOML

[webservers]
<YOUR_EC2_PUBLIC_IP> ansible_user=ec2-user ansible_ssh_private_key_file=~/.ssh/my-key-pair

Remember to replace:

<YOUR_EC2_PUBLIC_IP> with the actual Public IPv4 address of your EC2 instance.

my-key-pair.pem with the actual name of your private key file.

A quick explanation of these lines:

[webservers]: This defines a group of hosts. You can name your groups anything you like (e.g., db_servers, dev_machines). This makes it easy to target multiple servers at once.

<YOUR_EC2_PUBLIC_IP>: This is the actual address of your target machine.

ansible_user=ec2-user: This tells Ansible to use the ec2-user account to connect to the EC2 instance via SSH. (If you used an Ubuntu AMI, it might be ubuntu.)

ansible_ssh_private_key_file=~/.ssh/my-key-pair.pem: This tells Ansible which SSH private key to use for authentication when connecting to this host. ~/.ssh/my-key-pair.pem is the path where you copied your key in the previous step.
Save and exit nano:

Press Ctrl + O (that's the letter O, not zero) to write out the file.

Press Enter to confirm the filename.

Press Ctrl + X to exit the editor.

Now you have an inventory file telling Ansible about your first EC2 target. Let me know when this is done, and we'll run a quick test.

# Test Connectivity with the ansible Command

1.Open your WSL Ubuntu terminal (if it's not already open).

2.Navigate to your Ansible project directory:

Bash Command

------------------------->cd ~/my_aws_ansible_project
(This is important because your inventory.ini file is in this directory).

3.Run the following Ansible command:

Bash Command

---------------------->ansible webservers -i inventory.ini -m ping

# Let's break down this command:

ansible: This is the command-line tool for running ad-hoc Ansible commands (single tasks, not full playbooks).

webservers: This tells Ansible to run the command on all hosts that belong to the webservers group, as defined in your inventory.ini file.

-i inventory.ini: This specifies that Ansible should use your inventory.ini file to find the host information.

-m ping: This tells Ansible to use the ping module. The ping module simply checks if Ansible can connect to the remote host and run a Python script on it. It's like a health check for Ansible connectivity.

# What to expect as output:



If successful, you should see output similar to this:

<YOUR_EC2_PUBLIC_IP> | SUCCESS => {
    "ansible_facts": {
        "discovered_interpreter_python": "/usr/bin/python3"
    },
    "changed": false,
    "ping": "pong"
}
The key part here is SUCCESS => {"ping": "pong"}. This confirms that Ansible successfully connected to your EC2 instance, authenticated using your key, and was able to execute a basic task on it!

If you get any errors, stop there and let me know the exact error message. We can troubleshoot it together.

Once you see the "pong" message, let me know, and we'll move on to writing our first actual configuration script (an Ansible Playbook).

# Write Your First Ansible Playbook



Step 6: Write Your First Ansible Playbook
Now that connectivity is confirmed, we're ready to write a real configuration script, which Ansible calls a Playbook. Playbooks are written in YAML (YAML Ain't Markup Language) and define a set of tasks to be executed on your managed nodes.

For this first playbook, let's do something simple but tangible: we'll install the Nginx web server on your EC2 instance and make sure it's running.

For this first playbook, let's do something simple but tangible: we'll install the Nginx web server on your EC2 instance and make sure it's running.

1.In your WSL Ubuntu terminal, ensure you are still in your Ansible project directory:

#Bash Command

------------------->cd ~/my_aws_ansible_project

2.Create a new file for your playbook called install_nginx.yml:

#Bash Command

------------------->nano install_nginx.yml
This will open the nano text editor.

# Copy and paste the following content into the install_nginx.yml file:

In [4]:
---
- name: Configure Webserver with Nginx on Amazon Linux
  hosts: webservers
  become: true # This allows Ansible to use sudo on the remote machine
  tasks:
    - name: Install Nginx package (for RedHat/Amazon Linux)
      ansible.builtin.dnf:  # Changed from apt to dnf
        name: nginx
        state: present
        update_cache: yes # Similar to apt update_cache

    - name: Ensure Nginx service is running and enabled at boot
      ansible.builtin.service:
        name: nginx
        state: started
        enabled: yes

    - name: Create a simple index.html file
      ansible.builtin.copy:
        content: "<h1>Hello from Ansible! Nginx is working on Amazon Linux!</h1>"
        dest: /usr/share/nginx/html/index.html # Default Nginx HTML directory for Amazon Linux
        mode: '0644'

SyntaxError: invalid syntax (2842399209.py, line 1)

Let's quickly break down what this playbook does:

---: This is standard YAML syntax to indicate the start of a document.
    
- name: Configure Webserver with Nginx: A human-readable name for your playbook.
        
hosts: webservers: This tells Ansible to run this playbook on all hosts that are part of the webservers group in your inventory.ini file.
        
become: true: This is crucial! Most system administration tasks (like installing software or starting services) require root privileges. become: true tells Ansible to use sudo on the remote server to execute these tasks.
            
tasks:: This marks the beginning of the list of actions Ansible will perform.
        
- name: Update apt cache...: Each - name: line defines a specific task. This task updates the package list on your EC2 instance.

nsible.builtin.apt:: This is an Ansible module for managing packages on Debian/Ubuntu-based systems (like the Ubuntu or Amazon Linux 2 AMIs you might be using).
update_cache: yes: An argument to the apt module to refresh the package cache.
when: ansible_os_family == "Debian": This is a conditional statement. It ensures this task only runs if the operating system is identified as a Debian-family distribution (like Ubuntu). Amazon Linux is technically RedHat family, but this task is harmless if skipped there.
- name: Install Nginx package: Installs the nginx package. state: present ensures it's installed.
- name: Ensure Nginx service is running and enabled at boot: This task uses the service module to make sure Nginx is not only running (state: started) but also starts automatically after a reboot (enabled: yes).
- name: Create a simple index.html file: This task uses the copy module to place a basic index.html file in the web server's default directory.
content: The actual HTML content.
dest: The destination path on the EC2 instance.
mode: File permissions.

# Save and exit nano:

Press Ctrl + O (that's the letter O)

Press Enter to confirm the filename.

Press Ctrl + X to exit.

# Execute Your First Ansible Playbook

1.Open your WSL Ubuntu terminal (if it's not already open).

2.Make sure you are in your Ansible project directory:

Bash Command

------------------>cd ~/my_aws_ansible_project

Your inventory.ini and install_nginx.yml files should be in this directory.

3.Run the playbook using the ansible-playbook command:

Bash Command

------------------->ansible-playbook -i inventory.ini install_nginx.yml

Excellent! Your first Ansible playbook, install_nginx.yml, is now saved and ready.

Step 7: Execute Your First Ansible Playbook
This is the moment of truth! We'll now run the playbook to automate the installation and configuration of Nginx on your AWS EC2 instance.

Open your WSL Ubuntu terminal (if it's not already open).

Make sure you are in your Ansible project directory:

Bash Command

------------->cd ~/my_aws_ansible_project
Your inventory.ini and install_nginx.yml files should be in this directory.

Run the playbook using the ansible-playbook command:

Bash Command

---------->ansible-playbook -i inventory.ini install_nginx.yml
Let's break down this command:

ansible-playbook: This is the command used to execute Ansible playbooks.
-i inventory.ini: Just like with the ansible command, this tells ansible-playbook to use your inventory.ini file to find the hosts.
install_nginx.yml: This is the name of the playbook file you want to execute.
What to expect as output:

As the playbook runs, you'll see a series of tasks being executed. Each task will show its status (e.g., ok, changed, failed).

ok: Means the task ran successfully, and the target machine was already in the desired state (no changes were needed).
changed: Means the task ran successfully, and it made a change to the target machine to bring it to the desired state (e.g., installed Nginx, started the service).
failed: Means the task encountered an error.

# A successful run will look something like this (your specific IP and task names will vary):

In [6]:
PLAY [Configure Webserver with Nginx] ******************************************

TASK [Gathering Facts] *********************************************************
ok: [54.123.45.67]

TASK [Update apt cache (for Debian/Ubuntu-based systems)] **********************
changed: [54.123.45.67]

TASK [Install Nginx package] ***************************************************
changed: [54.123.45.67]

TASK [Ensure Nginx service is running and enabled at boot] *********************
changed: [54.123.45.67]

TASK [Create a simple index.html file] *****************************************
changed: [54.123.45.67]

PLAY RECAP *********************************************************************
54.123.45.67               : ok=5    changed=4    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0

SyntaxError: invalid syntax (600921415.py, line 1)

# Verify the Configuration

# Once the playbook completes with failed=0, it means Nginx should be installed and running on your EC2 instance. Let's verify it!

Open a web browser on your Windows 11 machine.

In the address bar, type the Public IPv4 address of your EC2 instance and press Enter.

You should now see a simple web page displaying:

"Hello from Ansible! Nginx is working!"

If you see this page, congratulations! You've successfully installed and configured a web server on an AWS EC2 instance using Ansible from your WSL environment. This is a significant achievement in automation.

# If you get any error Follow the below steps To check .

Step 1: Verify Nginx Status and Content on the EC2 Instance
    
    

SSH back into your EC2 instance from your WSL terminal:

Bash Command

-------------------------->ssh -i ~/.ssh/my-key-pair.pem ec2-user@<YOUR_EC2_PUBLIC_IP>

(Remember to replace my-key-pair.pem and <YOUR_EC2_PUBLIC_IP> with your actual values).

Srep2.Once you're inside the EC2 instance, run these commands:

Check Nginx service status:

Bash Command

--------------->systemctl status nginx

You should see output indicating Active: active (running). If it says inactive or failed, that's a problem.